In [2]:
# open key

f = open('secret.key')
key = f.read()
f.close()

In [9]:
# load dataframe

import pandas as pd
import requests
import json

df = pd.read_csv('../data/main_dataset.csv')

coords = {'progress': 0, 'coords': []}
try:
    f = open('check.txt')
    coords = eval(f.read())
    f.close()
except FileNotFoundError:
    coords = {'progress': 0, 'coords': []}

t = len(df) // 200
c = 0
c_all = coords['progress']
p = coords['progress'] // t

In [10]:
def find_coordinates(df_row):
    global t
    global c
    global p
    global c_all
    global coords
    if c == t:
        # checkpoint
        p += 1
        print(f"f{p / 2}% done geocoding")
        coords['progress'] = c_all
        f = open('check.txt', 'w')
        f.write(str(coords))
        f.close()
        c = 0

    addr = (df_row['ST_NUM']+' '+df_row['ST_NAME']+' '+df_row['CITY']+' '+df_row['ZIPCODE']).replace('\n', '')
    addr_nospace = addr.replace(' ', '%20')
    ret = None
    try:
        res = requests.post('https://maps.googleapis.com/maps/api/geocode/json?address=' + addr_nospace + '&key=' + key)
        res = res.json()
    except Exception as e:
        #print("Error", e, "while getting", addr)
        coords['coords'].append(('na', 'na'))
        ret = ('na', 'na')
    if res['status'] != 'OK':
        #print("No location for", addr)
        coords['coords'].append(('na', 'na'))
        ret = ('na', 'na')
    else:
        #print("Found location for", addr)
        lat = res['results'][0]['geometry']['location']['lat']
        long = res['results'][0]['geometry']['location']['lng']
        coords['coords'].append((lat, long))
        ret = (lat, long)
    c += 1
    c_all += 1
     
df.astype(str).iloc[coords['progress']:].apply(find_coordinates, axis=1)

f22.5% done geocoding


TypeError: write() argument must be str, not dict